In [4]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../source')
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

from BetaUtils import beta_hedging_ret, get_beta
from energy_functions import correlation, neg_returns, volatility
from Hedge import Hedge

import datetime

from time import time


## Generate a portfolio for each of the 40 quarters in the data set that have a low correlation with the S&P, and run 200 simulations for each of these portfolios.  We expect that the hedging should do next to nothing.

In [5]:
begin = ['-01-01', '-04-01', '-07-01', '-10-01']
end = ['-03-31', '-06-30', '-09-30', '-12-31']
end_2 = ['-12-31', '-03-31', '-06-30', '-09-30']
col_1 = []
col_2 = []
col_3 = []
col_4 = []

for year in range(2005, 2015):
    for date in begin:
        col_1.append(str(year) + date)
# col_1 = col_1[1:]
        
col_2.append('2005-12-31')
for year in range(2006, 2016):
    for date in end:
        col_2.append(str(year) + date)
col_2 = col_2[:-1]
        
col_3.append('2005-10-01')
for year in range(2006, 2016):
    for date in begin:
        col_3.append(str(year) + date)
col_3 = col_3[:-1]
        
for year in range(2006, 2016):
    for date in end:
        col_4.append(str(year) + date)  


In [ ]:
start = time()
count = 0
# window=60

with open('../../source/files/low_correlation_{}.txt'.format(count), 'w') as f:
    pass

for idx, years in enumerate(zip(col_1, col_2, col_3, col_4)):
    y_1, y_2, y_3, y_4 = years
        
    best_correlation = np.inf
    best_portfolio = None
    hedge = Hedge(begindate=y_1, enddate=y_4)
    hedge_2 = Hedge(begindate=y_1, enddate=y_2)
    for i in range(1000):
        # Generate and assign portfolio
        portfolio = hedge.generateRandomPort(size=50)
        hedge_2.portfolio = portfolio
        
        # Check returns
        spy = hedge.spy.values
        unhedged_returns = hedge.dollar_portfolio_sum.values
        
        corr = correlation(unhedged_returns,spy)
        if corr < best_correlation:
            best_portfolio = portfolio
            best_correlation = corr
    
    if idx % 5 == 0:
        print idx, best_correlation
        sys.stdout.flush()
    
    # Write results
    with open('../../source/portfolios_low_correlation.txt', 'a') as f:
        f.write('{}'.format(idx))
        for stock in best_portfolio:
            f.write(',{}'.format(stock))
        f.write('\n')
        
    for i in range(100):
        with open('../../source/files/low_correlation_{}.txt'.format(count), 'w') as f:
            f.write('HEADER\n')
            f.write('{},{},{},{},'.format(y_1, y_2, y_3, y_4))
            f.write('60,{},{},0.1,0.99,0.001,100,10000,0,1,0,0,0'.format(idx, idx))
        count += 1
        
print 'It took {} seconds'.format(time() - start)

0 0.520690455961
